In [2]:
import pandas as pd
from pandas_gbq import read_gbq
import logging
from pathlib import Path
from datetime import datetime

## Config

In [41]:
SQL_PATH = '../sql/'
OUTPUT_PATH = '/mnt/windows_mount/FILE/'

projectid = 'ts-official-website'
private_key = "../key/TS-Official-Website-d438f1c2b14d.json"

configuration = {
   'query': {
     "useQueryCache": True
   }
}

### Logging

In [4]:
pass

### SQL Loader

In [11]:
sql_path = Path(SQL_PATH)
list(sql_path.resolve().glob('*.sql'))

[PosixPath('/home/ts1050391/Dropbox/Taishin/taishin_works/ga_data_fetch/sql/GA_MAIN_HITS_EVENT_INFO.sql'),
 PosixPath('/home/ts1050391/Dropbox/Taishin/taishin_works/ga_data_fetch/sql/GA_MAIN_HITS_CUSTOM_METRICS.sql'),
 PosixPath('/home/ts1050391/Dropbox/Taishin/taishin_works/ga_data_fetch/sql/GA_MAIN_HITS.sql'),
 PosixPath('/home/ts1050391/Dropbox/Taishin/taishin_works/ga_data_fetch/sql/GA_MAIN_CUSTOM_DIMENSIONS.sql'),
 PosixPath('/home/ts1050391/Dropbox/Taishin/taishin_works/ga_data_fetch/sql/GA_MAIN_HITS_PAGE.sql'),
 PosixPath('/home/ts1050391/Dropbox/Taishin/taishin_works/ga_data_fetch/sql/GA_MAIN_HITS_SOCIAL.sql'),
 PosixPath('/home/ts1050391/Dropbox/Taishin/taishin_works/ga_data_fetch/sql/GA_MAIN.sql'),
 PosixPath('/home/ts1050391/Dropbox/Taishin/taishin_works/ga_data_fetch/sql/GA_MAIN_TRAFFIC_SOURCE.sql'),
 PosixPath('/home/ts1050391/Dropbox/Taishin/taishin_works/ga_data_fetch/sql/GA_MAIN_TOTALS.sql'),
 PosixPath('/home/ts1050391/Dropbox/Taishin/taishin_works/ga_data_fetch/sql/GA

In [45]:
sql_files = sql_path.resolve().glob('*.sql')

sqls = []
for _, file_path in enumerate(sql_files):
    with open(file_path, 'r', encoding='utf8') as file:
        try:
            sqls.append(file.read())
        except Exception as e:
            print(e)

In [7]:
len(sqls)

18

### get_bq_result

In [54]:
sql = sqls[17].format(start_date='20180314', end_date='20180314')
print(sql)

--GA_MAIN_HITS_CUSTOM_VARIABLES--
--行銷平台--

SELECT TO_HEX(SHA256(Concat(cast(FullvisitorID as string),'_',cast(VISITID as string)))) AS GA_MAIN_NO,
	   FullvisitorID as FULL_VISITOR_ID ,visitID as VISIT_ID ,h.hitNumber as HIT_NUMBER ,
	   hcv.index AS INDEX, 
	   hcv.customVarName AS CUSTOM_VAR_NAME, 
	   hcv.customVarValue AS CUSTOM_VAR_VALUE
FROM `ts-official-website.164284491.ga_sessions_*`,
	   UNNEST(hits) AS h,
	   UNNEST(h.customVariables) AS hcv
WHERE _TABLE_SUFFIX BETWEEN '20180314' and '20180314'


In [49]:
df = read_gbq(
    sql,
    private_key=private_key,
    project_id=projectid,
    configuration=configuration,
    dialect='standard'
)


In [52]:
df

,GA_MAIN_NO,FULL_VISITOR_ID,VISIT_ID,HIT_NUMBER,INDEX,CUSTOM_VAR_NAME,CUSTOM_VAR_VALUE


In [8]:
def save_bq_csv_oneday(sql, data_date, output):
    
    df = read_gbq(
        sql.format(start_date=data_date, end_date=data_date),
        private_key=private_key,
        project_id=projectid,
        configuration=configuration,
        dialect='standard'
    )
    
    df.to_csv(output, index=false)
    
    return True

In [ ]:
def _dates_between(start_date, end_date):
    start_date = datetime.strptime(start_date)
    end_date = datetime.strptime(end_date)
    
    

In [ ]:
def save_bq_csv(sql, start_date, end_date, output):
    
     
    
    
    

### Run

In [51]:
print(sqls[17])

--GA_MAIN_HITS_CUSTOM_VARIABLES--
--行銷平台--

SELECT TO_HEX(SHA256(Concat(cast(FullvisitorID as string),'_',cast(VISITID as string)))) AS GA_MAIN_NO,
	   FullvisitorID as FULL_VISITOR_ID ,visitID as VISIT_ID ,h.hitNumber as HIT_NUMBER ,
	   hcv.index AS INDEX, 
	   hcv.customVarName AS CUSTOM_VAR_NAME, 
	   hcv.customVarValue AS CUSTOM_VAR_VALUE
FROM `ts-official-website.164284491.ga_sessions_*`,
	   UNNEST(hits) AS h,
	   UNNEST(h.customVariables) AS hcv
WHERE _TABLE_SUFFIX BETWEEN '{start_date}' and '{end_date}'


In [ ]:
save_bq_csv()